# TP 3 : Resolution dynamique applied knapsack problem

### Initialisation (à faire une seule fois)

In [10]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

   Resolving package versions...


  No Changes to `C:\Users\ZOUICHA MOHAMMED\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\ZOUICHA MOHAMMED\.julia\environments\v1.9\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\ZOUICHA MOHAMMED\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\ZOUICHA MOHAMMED\.julia\environments\v1.9\Manifest.toml`


### Récupération des données

In [11]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [12]:
function testSondability_LP(affichage,UB,LB,list_xi,poids,listobjs, listvals,capacity, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    capacite1 = capacity
    for i in 1:length(listobjs)
        if listvals[i] == 1
            capacite1 -= poids[listobjs[i]]
        end
    end
    
    if(capacite1 < 0)#Test de faisabilite:le noeud a une capacité négative
        TA=true
        if affichage
            println("TA")
        end
    #Test d'optimalite:la borne supérieure est inférieure à la meilleure solution connue
    elseif(UB <= BestProfit) 
        TO=true
        if affichage
            println("TO")
        end
    elseif UB == LB
         #Test de resolution:la borne supérieure et inférieure sont égale
        TR=true
        if affichage
            println("TR")
        end
        if ((UB >= BestProfit))
            Bestsol = list_xi
            BestProfit=UB
            if affichage
                println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
            end
        end
    else
        if affichage
            println("non sondable")
        end
    end
    TA, TO, TR, Bestsol, BestProfit
end

testSondability_LP (generic function with 2 methods)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [13]:
Pkg.add("Clp");
Pkg.add("JuMP");
using JuMP, Clp #to comment for the lab class

function updateModele_LP!(model2, x, listobjs, listvals)
    for i in 1:length(listobjs)
        set_lower_bound(x[listobjs[i]],listvals[i])
        set_upper_bound(x[listobjs[i]],listvals[i])
    end
end

function reset_LP!(model2, x, listobjs)
    for i in 1:length(listobjs)
        set_lower_bound(x[listobjs[i]],0.0)
        set_upper_bound(x[listobjs[i]],1.0)
    end
end

function resetAll_LP!(model2, x)
    for i in 1:length(x)
        set_lower_bound(x[i],0.0)
        set_upper_bound(x[i],1.0)
    end
end

function createModel_LP(price, weight, capacity)
# ROOT NODE

    n=length(price)

    model2 = Model(Clp.Optimizer) # set optimizer
    set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
    set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

    # define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
    @variable(model2, 0 <= x[i in 1:n] <= 1)

    # define objective function
    @objective(model2, Max, sum(price[i]*x[i] for i in 1:n))

    # define the capacity constraint
    @constraint(model2, sum(weight[i]*x[i] for i in 1:n) <=  capacity)

    println(model2)

    return model2, x
end

   Resolving package versions...


  No Changes to `C:\Users\ZOUICHA MOHAMMED\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\ZOUICHA MOHAMMED\.julia\environments\v1.9\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\ZOUICHA MOHAMMED\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\ZOUICHA MOHAMMED\.julia\environments\v1.9\Manifest.toml`

createModel_LP (generic function with 1 method)

### Resolution Dynamique 


In [16]:
function Tab(prix, poids, capacite)
    tab = fill(0, (0:length(prix) , 0:capacite))
    for i in 1:length(prix)
    
        for j in 0: (capacite)
      
            if j == 0
                tab[i,j] = 0 

            elseif (poids[i]>j)
                tab[i,j] = tab[i-1,j]
            else 
                tab[i,j] = max(tab[i-1,j], tab[i-1,j-poids[i]]+prix[i])
            end 
        end 
    end
    return tab
end

function ResolutionDynamique(tab, prix, poids, capacite)
    Bestsol = zeros(length(prix))
    BestProfit = 0 

    i = length(prix)
    j = capacite
    BestProfit= tab[i,j]
    while (i > 1)
        if tab[i-1,j] == tab[i,j]
            Bestsol[i] = 0.0
        else 
            i_choisi = i
            Bestsol[i_choisi] = 1.0
            j = j - poids[i_choisi]
            #if j <= 0
            #    j = 1
            #end 
        end 
        i-=1
    end 

    if tab[i,j] != 0
        Bestsol[i] = 1.0
    end 

    return Bestsol, BestProfit
end 

function SolveKnapInstance1(filename)

    prix, poids, capacite = readKnaptxtInstance(filename)

    println("The prices are: ", join(string.(prix), ", "))
    println("The weights are: ", join(string.(poids), ", "))
    println("The capacity is :", capacite)

   # println("\n ***Resolution Dynamique :**** \n")

    tab = Tab(prix, poids, capacite)
    Bestsol, BestProfit = ResolutionDynamique(tab, prix, poids, capacite)

    #println("\n******\n\nOptimal value = ", BestProfit,"\n******\n\nsolution optimale =",Bestsol)
    
    return BestProfit, Bestsol
end 


SolveKnapInstance1 (generic function with 1 method)

### Affichage du résultat final

In [17]:
BestProfit, Bestsol = SolveKnapInstance1("InstancesKnapSack/test.opb.txt")
println("\n******\n\nOptimal value avec la resolution Dynamique = ", BestProfit, "\n\nOptimal x=", Bestsol)


The prices are: 42, 40, 12, 25
The weights are: 7, 4, 3, 5
The capacity is :10

******

Optimal value avec la resolution Dynamique = 65

Optimal x=[0.0, 1.0, 0.0, 1.0]


## Question 1 : Expliquer ce qu’est la programmation dynamique et la relation de récurrence associée à un problème de programmation dynamique

La programmation dynamique utilise des relations de récurrence pour résoudre des problèmes en les décomposant en sous-problèmes plus simples. La relation de récurrence décrit comment la solution globale peut être construite à partir des solutions des sous-problèmes.

## Question 3 : Donner une courte argumentation de l’adéquation du résultat avec l’instance résolue 

Cette relation reflète la nature optimale du choix en considérant chaque objet individuellement tout en maintenant la capacité du sac à dos.

La fonction $C_{i,j}$ représente la valeur maximale que l'on peut obtenir avec les $i$ premiers objets et une capacité de sac à dos de $j$. La relation de récurrence indique que cette valeur maximale peut être obtenue en considérant deux choix possibles :

1. **Ne pas inclure l'objet $i$:**
   - La valeur maximale pour les $i$ premiers objets et une capacité de sac à dos de $j$ est simplement $C_{i−1,j}$, c'est-à-dire la valeur maximale obtenue sans inclure l'objet $i$.

2. **Inclure l'objet $i$:**
   - La valeur maximale pour les $i$ premiers objets et une capacité de sac à dos de $j$ peut également être obtenue en ajoutant la valeur de l'objet $i$ ($c_i$) à la valeur maximale obtenue pour les $i-1$ premiers objets et une capacité de sac à dos réduite de la capacité requise pour l'objet $i$ ($w_i$). Cela donne $C_{i−1,j−w_i}+c_i$.

Ainsi, pour déterminer la valeur maximale pour les $i$ premiers objets et une capacité de sac à dos de $j$, on choisit le maximum entre les deux options ci-dessus :

$C_{i, j}=\max \left\{C_{i-1, j}, C_{i-1, j-w_i}+C_i\right\}$

En répétant ce processus pour chaque objet, l'algorithme explore toutes les combinaisons possibles d'inclusion ou d'exclusion d'objets pour obtenir la meilleure valeur tout en respectant la capacité du sac à dos. Cela garantit que la solution obtenue est optimale par rapport à la valeur des objets tout en respectant les contraintes de capacité. L'approche dynamique permet d'éviter des calculs redondants en mémorisant les résultats intermédiaires, améliorant ainsi l'efficacité de l'algorithme.

## Question 4 : Expliquer le fonctionnement de votre algorithme

Ces deux fonctions implémentent une solution en programmation dynamique pour le problème du sac à dos.
-> Tab(prix, poids, capacite) : Cette fonction crée et remplit un tableau de manière itérative en utilisant la programmation dynamique pour résoudre le probleme, en se basant sur la relation de récurrence : 

$C_{i, j}=\max \left\{C_{i-1, j}, C_{i-1, j-w_i}+C_i\right\}$

-> ResolutionDynamique(tab, prix, poids, capacite) : Cette fonction prend en entrée le tableau résultant de la phase de construction (tab), ainsi que les tableaux prix, poids et la capacité maximale du sac à dos. Elle utilise le tableau tab pour déduire la solution optimale tab[length(prix),capacite] , en indiquant quels objets doivent être inclus dans le sac à dos pour maximiser le profit. 

## Question 5 : Tests de l'algorithme




In [18]:
tests = ["InstancesKnapSack/almost_strongly_correlated/knapPI_5_100_1000_1_-2295.opb.txt",
"InstancesKnapSack/circle/knapPI_16_100_1000_1_-4400.opb.txt",
"InstancesKnapSack/inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt",
"InstancesKnapSack/inverse_strongly_correlated/knapPI_4_100_1000_1_-997.opb.txt",
"InstancesKnapSack/multiple_strongly_correlated/knapPI_14_20_1000_1_-1794.opb.txt",
"InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt",
"InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_3_-1002.opb.txt",
"InstancesKnapSack/similar_weights/knapPI_9_100_1000_1_-995.opb.txt",
"InstancesKnapSack/subset_sum/knapPI_6_100_1000_1_-997.opb.txt",
"InstancesKnapSack/uncorrelated/knapPI_1_100_1000_1_-9147.opb.txt",
"InstancesKnapSack/strongly_correlated_span/knapPI_13_500_1000_1_-3198.opb.txt",
"InstancesKnapSack/uncorrelated_span/knapPI_11_20_1000_1_-1428.opb.txt",
"InstancesKnapSack/weakly_correlated/knapPI_2_100_1000_1_-1514.opb.txt",
"InstancesKnapSack/weakly_correlated_span/knapPI_12_100_1000_1_-970.opb.txt"
]

for test in tests 
    debut = time()
    printstyled("\ntest  : ",test, ":\n", bold=true, color=:green)
    BestProfit, Bestsol = SolveKnapInstance1(test)
    fin = time()
    temps_calcul = fin-debut

 
    println("--- temps de calcul : ",temps_calcul)
    println("--- Optimal value = ", BestProfit, "\n--- Optimal x=", Bestsol)

end


test  : InstancesKnapSack/almost_strongly_correlated/knapPI_5_100_1000_1_-2295.opb.txt:
The prices are: 585, 426, 347, 521, 423, 894, 142, 946, 1054, 353, 108, 1001, 221, 194, 839, 673, 815, 983, 466, 1083, 398, 534, 781, 173, 973, 239, 957, 244, 1096, 630, 299, 377, 197, 820, 343, 208, 223, 170, 198, 699, 744, 366, 1071, 996, 313, 848, 586, 1023, 128, 773, 639, 653, 566, 146, 809, 654, 292, 824, 829, 1087, 191, 439, 650, 295, 965, 779, 669, 1037, 822, 752, 222, 531, 607, 684, 952, 741, 1092, 824, 386, 911, 958, 762, 189, 278, 287, 718, 360, 945, 293, 360, 613, 986, 631, 950, 395, 898, 492, 430, 398, 890
The weights are: 485, 326, 248, 421, 322, 795, 43, 845, 955, 252, 9, 901, 122, 94, 738, 574, 715, 882, 367, 984, 299, 433, 682, 72, 874, 138, 856, 145, 995, 529, 199, 277, 97, 719, 242, 107, 122, 70, 98, 600, 645, 267, 972, 895, 213, 748, 487, 923, 29, 674, 540, 554, 467, 46, 710, 553, 191, 724, 730, 988, 90, 340, 549, 196, 865, 678, 570, 936, 722, 651, 123, 431, 508, 585, 853, 642, 9

0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

test  : InstancesKnapSack/inverse_strongly_correlated/knapPI_4_100_1000_1_-997.opb.txt:
The prices are: 485, 94, 326, 506, 248, 416, 421, 992, 322, 649, 795, 237, 43, 457, 845, 815, 955, 446, 252, 422, 9, 791, 901, 359, 122, 667, 94, 598, 738, 7, 574, 544, 715, 334, 882, 766, 367, 994, 984, 893, 299, 633, 433, 131, 682, 428, 72, 700, 874, 617, 138, 874, 856, 720, 145, 419, 995, 794, 529, 196, 199, 997, 277, 116, 97, 908, 719, 539, 242, 707, 107, 569, 122, 537, 70, 931, 98, 726, 600, 487, 645, 772, 267, 513, 972, 81, 895, 943, 213, 58, 748, 303, 487, 764, 923, 536, 29, 724, 674, 789
The weights are: 585, 194, 426, 606, 348, 516, 521, 1092, 422, 749, 895, 337, 143, 557, 945, 915, 1055, 546, 352, 522, 109, 891, 1001, 459, 222, 767, 194, 698, 

0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]

test  : InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_3_-1002.opb.txt:
The prices are: 96, 507, 417, 993, 651, 237, 459, 816, 447, 423, 792, 360, 669, 600, 9, 546, 336, 768, 996, 894, 948, 819, 420, 801, 408, 942, 72, 561, 795, 585, 597, 696, 465, 744, 171, 354, 858, 183, 888, 606, 447, 483, 426, 609, 516, 999, 36, 306, 495, 744, 402, 33, 612, 537, 336, 810, 735, 600, 429, 966
The weights are: 94, 506, 416, 992, 649, 237, 457, 815, 446, 422, 791, 359, 667, 598, 7, 544, 334, 766, 994, 893, 946, 819, 420, 800, 406, 941, 70, 560, 795, 583, 595, 695, 463, 742, 171, 352, 858, 182, 887, 606, 447, 483, 425, 609, 514, 998, 34, 304, 495, 743, 400, 32, 612, 535, 336, 809, 735, 599, 427, 966
The capacity is :998
--- temps de calcul : 0.0019998550415039062
--- Optimal value = 1011
--- Optimal x=[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,

--- temps de calcul : 0.15799999237060547
--- Optimal value = 995
--- Optimal x=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

test  : InstancesKnapSack/subset_sum/knapPI_6_100_1000_1_-997.opb.txt:
The prices are: 485, 94, 326, 506, 248, 416, 421, 992, 322, 649, 795, 237, 43, 457, 845, 815, 955, 446, 252, 422, 9, 791, 901, 359, 122, 667, 94, 598, 738, 7, 574, 544, 715, 334, 882, 766, 367, 994, 984, 893, 299, 633, 433, 131, 682, 428, 72, 700, 874, 617, 138, 874, 856, 720, 145, 419, 995, 794, 529, 196, 199, 997, 277, 116, 97, 908, 719, 539, 

0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

test  : InstancesKnapSack/strongly_correlated_span/knapPI_13_500_1000_1_-3198.opb.txt:
The prices are: 234, 39, 1053, 351, 585, 78, 117, 234, 312, 156, 156, 273, 351, 351, 312, 468, 156, 585, 468, 1053, 702, 1053, 819, 234, 39, 1170, 234, 234, 351, 936, 468, 351, 351, 156, 351, 390, 468, 156, 312, 390, 468, 702, 234, 1170, 195, 117, 351, 936, 585, 78, 702, 234, 117, 117, 117, 819, 234, 702, 702, 156, 39, 351, 1170, 117, 156, 390, 390, 78, 1053, 936, 312, 156, 312, 39, 351, 39, 819, 585, 234, 585, 1170, 468, 195, 3

...

0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

test  : InstancesKnapSack/uncorrelated_span/knapPI_11_20_1000_1_-1428.opb.txt:
The prices are: 114, 38, 133, 95, 612, 171, 918, 408, 714, 510, 114, 76, 57, 204, 1020, 816, 190, 510, 114, 714
The weights are: 582, 194, 679, 485, 396, 873, 594, 264, 462, 330, 582, 388, 291, 132, 660, 528, 970, 330, 582, 462
The capacity is :970
--- temps de calcul : 0.0010001659393310547
--- Optimal value = 1428
--- Optimal x=[0.0, 0.0, 0.0, 0.

485, 326, 248, 421, 322, 795, 43, 845, 955, 252, 9, 901, 122, 94, 738, 574, 715, 882, 367, 984, 299, 433, 682, 72, 874, 138, 856, 145, 995, 529, 199, 277, 97, 719, 242, 107, 122, 70, 98, 600, 645, 267, 972, 895, 213, 748, 487, 923, 29, 674, 540, 554, 467, 46, 710, 553, 191, 724, 730, 988, 90, 340, 549, 196, 865, 678, 570, 936, 722, 651, 123, 431, 508, 585, 853, 642, 992, 725, 286, 812, 859, 663, 88, 179, 187, 619, 261, 846, 192, 261, 514, 886, 530, 849, 294, 799, 391, 330, 298, 790
The capacity is :995
--- temps de calcul : 0.04200005531311035
--- Optimal value = 1514
--- Optimal x=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## Question 6 : Comparer les temps de calcul obtenus avec votre algorithme de programmation dynamique et votre branch-and-bound implémenté lors du TP2 

 Le resultat comprend des instances de test, leurs prix, poids, valeurs optimales et les temps de calcul pour la programmation dynamique et le branch-and-bound. Les résultats indiquent que la programmation dynamique est plus rapide pour les grandes instances et les objets non corrélés, car elle évite de recalculer les mêmes scénarios. Le branch-and-bound fonctionne bien pour des objets fortement corrélés, réduisant rapidement l'espace de recherche.

In [20]:
## Resolution Branch & Bound 
struct Noeud
    vars::Vector{Int}     # Les objets choisis pour ce noeud
    vals::Vector{Float64} # Les valeurs correspondantes 
    ub::Float64           # La borne sup de ce noeud
    lb::Float64           # La borne inf de ce noeud
    num_noued_pere::Int64 # le num du noeud parent de ce noeud
end

function separateNodeThenchooseNext_bestUB!(noeud_actuel::Noeud, list_best_ratio, price, weight, capacity,borne,num_noued_pere)
    if borne == "borne1"
        # Pour la borne1, choisir l'objet avec le meilleur ratio non encore pris
        obj = 0
        for i in list_best_ratio
            if (!(i in noeud_actuel.vars) && (obj==0))
                obj = i
            end
        end
    elseif borne == "borne2"
        # Pour la borne2, choisir l'objet la plus fractionnaire
        _,_,obj = Borne2(capacity, weight,price,  noeud_actuel.vals, noeud_actuel.vars, list_best_ratio)
    end    
    # Créer deux nouveaux noeuds
    
    # noeud gauche
    noeud_gauche_inter = Noeud(copy(noeud_actuel.vars), copy(noeud_actuel.vals), noeud_actuel.ub, noeud_actuel.lb,num_noued_pere)
    push!(noeud_gauche_inter.vars, obj)
    push!(noeud_gauche_inter.vals, 1.0)
    
    # Calculer les bornes sup et inf pour le noeud gauche
    if borne == "borne1"
        UB,LB,_ = Borne1(capacity, weight,price,  noeud_gauche_inter.vals, noeud_gauche_inter.vars, list_best_ratio)
    elseif borne == "borne2"
        UB,LB,_,_ = Borne2(capacity, weight,price,  noeud_gauche_inter.vals, noeud_gauche_inter.vars, list_best_ratio)
    end
    noeud_gauche = Noeud(copy(noeud_gauche_inter.vars), copy(noeud_gauche_inter.vals), UB, LB,num_noued_pere)


    # noeud droit
    noeud_droit_inter =  Noeud(copy(noeud_actuel.vars), copy(noeud_actuel.vals), noeud_actuel.ub, noeud_actuel.lb,num_noued_pere)
    push!(noeud_droit_inter.vars, obj)
    push!(noeud_droit_inter.vals, 0.0)
    
    # Calculer les bornes sup et inf pour le noeud droit
    if borne == "borne1"
        UB,LB,_  = Borne1(capacity, weight,price, noeud_droit_inter.vals, noeud_droit_inter.vars, list_best_ratio)
    elseif borne == "borne2"
        UB,LB,_ ,_ = Borne2(capacity, weight,price, noeud_droit_inter.vals, noeud_droit_inter.vars, list_best_ratio)
    end
    noeud_droit = Noeud(copy(noeud_droit_inter.vars), copy(noeud_droit_inter.vals), UB, LB,num_noued_pere)
    
    return noeud_gauche, noeud_droit
end

function exploreNextNode_bestUB!(list_des_noeuds)

    # Si aucun noeud n'est à explorer, retourner true pour signaler la fin
    if isempty(list_des_noeuds)
        return true, nothing  
    else
        # Rechercher le noeud avec la plus grande borne supérieure
        max_ub = -Inf
        noeud_suivant = nothing
        for noeud in list_des_noeuds
            if noeud.ub > max_ub
                max_ub = noeud.ub
                noeud_suivant = noeud
            end
        end
        deleteat!(list_des_noeuds, findfirst(==(noeud_suivant), list_des_noeuds))
        return false, noeud_suivant
    end
end


exploreNextNode_bestUB! (generic function with 1 method)

In [21]:
function Borne1(capacite, poids, cout, listvals, listobjs, list_best_ratios)
    prixtotal = 0 #prix total des objets pris jusqu'à présent
    capacite1 = capacite #capacite restante
    
    n = length(listobjs)
    list_xi = [0.0 for i in 1:length(poids)] # Liste pour conserver les valeurs des objets pris
    
    # Si aucun objet n'a été pris, utilisez l'objet avec le meilleur ratio pour calculer les bornes
    if n == 0
        i_max = list_best_ratios[1]
        UB = capacite * (cout[i_max] / poids[i_max])
        LB = 0
        list_xi[i_max] = capacite / poids[i_max]
        return UB,LB,list_xi
    end
     
    # sinon on calcul la capacité restante,prix total et de la liste des proportions pour les objets déjà pris
    for i in 1:length(listobjs)
        if listvals[i] == 1
            list_xi[listobjs[i]] = 1
            capacite1 -= poids[listobjs[i]]
            prixtotal += cout[listobjs[i]]
        end
    end
    
    # Trouver l'objet non pris avec le meilleur ratio et calculer sa proportion qui peut être prise
    i_max = -1
    toto = 0 # Quantité de l'objet avec le meilleur ratio qui peut être prise
    for i in list_best_ratios
        if !(i in listobjs)
            i_max = i
            toto = capacite1/poids[i_max]
            list_xi[i_max]=toto
            break;

        end
    end
    if (n == length(poids)) 
            UB = prixtotal
    else 
        UB = prixtotal + toto*cout[i_max]
    end
    LB = prixtotal
    return UB,LB,list_xi
end

# Calcule la liste des indices des objets triés par le meilleur ratio
function best_ratio(price, weight) 
    ratio = [(price[i]/weight[i],i) for i in 1:length(price)]
    sort!(ratio,by = x -> x[1], rev=true)
    return [i for (_,i) in ratio]
end

Borne1(10, [7, 4, 3, 5], [42, 40, 12, 25], [1], [2], [2, 1, 4, 3])

(76.0, 40, [0.8571428571428571, 1.0, 0.0, 0.0])

In [22]:
function Borne2(capacite, poids, cout, listvals, listobjs, list_best_ratios)
    prixtotal = 0  #prix total des objets pris jusqu'à présent
    capacite1 = capacite #capacite restante
    n = length(listobjs)
    list_xi = [0.0 for i in 1:length(poids)] # Liste pour conserver les valeurs des objets pris
    
    # Si aucun objet n'a été pris
    if n == 0
        UB = 0
        LB = 0
        elem_fract = 1
        # On parcourt les objets dans l'ordre du meilleur ratio jusqu'à ce que la capacité soit atteinte.
        for i in list_best_ratios
            # Si l'objet entier peut être pris
            if (capacite1 - poids[i] >= 0)
                capacite1 = capacite1 - poids[i]
                list_xi[i] = 1
                UB = UB + cout[i]
                LB = LB + cout[i]
            # Sinon on prend une portion et c'est lui l'element le plus fractionnaire
            elseif (capacite1  > 0)
                UB = UB + cout[i]*(capacite1/poids[i])
                list_xi[i] = (capacite1/poids[i])
                capacite1 = 0
                elem_fract = i
            end
        end
        
        return UB,LB,elem_fract ,list_xi
    end
    
    # sinon on calcul prixtotal et capacite restante   
    for i in 1:length(listobjs)
        if listvals[i] == 1
            list_xi[listobjs[i]] = 1
            capacite1 -= poids[listobjs[i]]
            prixtotal += cout[listobjs[i]]
        end
    end
    
    LB = prixtotal
    UB = prixtotal
    elem_fract = 1
    # On parcourt les objets dans l'ordre du meilleur ration qui ne sont pas encore pris
    # jusqu'à ce que la capacité soit atteinte.
    for i in list_best_ratios
        if !(i in listobjs)
            # Si l'objet entier peut être pris
            if (capacite1 - poids[i] >= 0)
                capacite1 = capacite1 - poids[i]
                list_xi[i] = 1
                UB = UB + cout[i]
                LB = LB + cout[i]
            # Sinon on prend une portion et c'est lui l'element le plus fractionnaire
            elseif (capacite1  > 0)
                UB = UB + cout[i]*(capacite1/poids[i]) 
                list_xi[i] = (capacite1/poids[i])
                elem_fract = i
                capacite1 = 0
            end
        end
    end
    
    return UB,LB,elem_fract,list_xi
end
Borne2(14, [5;7;4;3], [16;22;12;8], [0], [0], [1;2;3;4])

(44.0, 38, 3, [1.0, 1.0, 0.5, 0.0])

In [23]:
function solveKnapInstance(filename,borne,affichage)
    price, weight, capacity = readKnaptxtInstance(filename)
    
    # Initialisation des structures de données
    nodes = [Noeud([], [], Inf,0,0)]  # Commencez avec le noeud racine
    BestProfit = -1.0
    Bestsol = Float64[]
    
    trParentnodes = Int64[]
    trChildnodes = Int64[]
    trNamenodes = []

    current_node_number = 0
    stop = false

    while !stop
        if affichage 
            println("\nNode number ", current_node_number, ": \n---------------\n")
        end
        # Sélection et mise à jour du nœud actuel à partir de la liste des noeuds
        stop, current_node = exploreNextNode_bestUB!(nodes)  # Déterminez le noeud suivant avec la plus grande BS
        if stop
            break  # Sortir du boucle si aucun noeud n'est à explorer
        end

        #liste des obj et valeur pour ce noeud
        listobjs = current_node.vars
        listvals = current_node.vals

        #Update the graphical tree
        push!(trNamenodes, current_node_number +1)
        if length(trNamenodes) >= 2
            push!(trParentnodes, current_node.num_noued_pere + 1)
            push!(trChildnodes, current_node_number+1)
        end
        
        if affichage
            println("\n******\n\nlistobjs = ", listobjs, "\n\nlistvals =", listvals)
        end
        
        # Calcul de la borne sup et inf pour ce noeud
        if (borne == "borne1")
            UB,LB,list_xi = Borne1(capacity,weight, price, listvals, listobjs, best_ratio(price, weight))
        elseif (borne == "borne2")
            UB,LB,_,list_xi = Borne2(capacity,weight, price, listvals, listobjs, best_ratio(price, weight))
        end
        
        if affichage
            println("\n******\n\nUB = ", UB, "\n\nLB =", LB)
        end
        
        # Déterminer si le noeud est sondable
        TA , TO , TR,Bestsol,BestProfit = testSondability_LP(affichage,UB,LB,list_xi,weight,listobjs, listvals,capacity, BestProfit, Bestsol)
        is_node_sondable = TA || TO || TR 

        if !is_node_sondable
            node_including_obj, node_excluding_obj = separateNodeThenchooseNext_bestUB!(current_node, best_ratio(price, weight), price, weight, capacity,borne,current_node_number)
            push!(nodes, node_including_obj)
            push!(nodes, node_excluding_obj)
        end

        current_node_number += 1
    end
    if affichage
        println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
    end
    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes,current_node_number
end
INSTANCE = "InstancesKnapSack/test.opb.txt"
solveKnapInstance(INSTANCE,"borne2",false)


## On compare les resultats issues des deux resoltions dynamique et branch & bound 
tests = [
"InstancesKnapSack/uncorrelated_span/knapPI_11_20_1000_1_-1428.opb.txt",
"InstancesKnapSack/almost_strongly_correlated/knapPI_5_100_1000_1_-2295.opb.txt",
"InstancesKnapSack/circle/knapPI_16_100_1000_1_-4400.opb.txt",
"InstancesKnapSack/inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt",
"InstancesKnapSack/inverse_strongly_correlated/knapPI_4_100_1000_1_-997.opb.txt",
"InstancesKnapSack/similar_weights/knapPI_9_100_1000_1_-995.opb.txt",
"InstancesKnapSack/subset_sum/knapPI_6_100_1000_1_-997.opb.txt",
"InstancesKnapSack/uncorrelated/knapPI_1_100_1000_1_-9147.opb.txt",
"InstancesKnapSack/weakly_correlated/knapPI_2_100_1000_1_-1514.opb.txt",
"InstancesKnapSack/weakly_correlated_span/knapPI_12_100_1000_1_-970.opb.txt",
"InstancesKnapSack/multiple_strongly_correlated/knapPI_14_20_1000_1_-1794.opb.txt",
"InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt",
"InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_3_-1002.opb.txt"
]

for test in tests 
    printstyled("\ntest  : ",test, ":\n\n", bold=true, color=:green)
    debut = time()
    BestProfit, Bestsol = SolveKnapInstance1(test)

    fin = time()
    temps_calcul_dynamique = fin-debut

    debut = time()
    BestProfit, Bestsol = solveKnapInstance(test,"borne2",false)
    fin = time()
    temps_calcul_bab_1 = fin-debut
    
    debut = time()
    BestProfit, Bestsol = solveKnapInstance(test,"borne1",false)
    fin = time()
    temps_calcul_bab_2 = fin-debut
    
    println("--- temps de calcul (Programmation Dynamique) : ",temps_calcul_dynamique)
    
    println("\n--- temps de calcul (Branch & Bound: Borne 1) : ",temps_calcul_bab_1)
    
    println("\n--- temps de calcul (Branch & Bound: Borne 2) : ",temps_calcul_bab_2)

end


test  : InstancesKnapSack/uncorrelated_span/knapPI_11_20_1000_1_-1428.opb.txt:

The prices are: 114, 38, 133, 95, 612, 171, 918, 408, 714, 510, 114, 76, 57, 204, 1020, 816, 190, 510, 114, 714
The weights are: 582, 194, 679, 485, 396, 873, 594, 264, 462, 330, 582, 388, 291, 132, 660, 528, 970, 330, 582, 462
The capacity is :970
--- temps de calcul (Programmation Dynamique) : 0.0019998550415039062

--- temps de calcul (Branch & Bound: Borne 1) : 0.015000104904174805

--- temps de calcul (Branch & Bound: Borne 2) : 0.0009999275207519531

test  : InstancesKnapSack/almost_strongly_correlated/knapPI_5_100_1000_1_-2295.opb.txt:

The prices are: 585, 426, 347, 521, 423, 894, 142, 946, 1054, 353, 108, 1001, 221, 194, 839, 673, 815, 983, 466, 1083, 398, 534, 781, 173, 973, 239, 957, 244, 1096, 630, 299, 377, 197, 820, 343, 208, 223, 170, 198, 699, 744, 366, 1071, 996, 313, 848, 586, 1023, 128, 773, 639, 653, 566, 146, 809, 654, 292, 824, 829, 1087, 191, 439, 650, 295, 965, 779, 669, 1037, 822, 


--- temps de calcul (Programmation Dynamique) : 0.015999794006347656

--- temps de calcul (Branch & Bound: Borne 1) : 0.05300021171569824

--- temps de calcul (Branch & Bound: Borne 2) : 0.05799984931945801

test  : InstancesKnapSack/circle/knapPI_16_100_1000_1_-4400.opb.txt:

The prices are: 403, 886, 814, 1151, 983, 629, 848, 1074, 839, 819, 1062, 762, 994, 950, 111, 914, 737, 1049, 1152, 1110, 973, 474, 824, 1013, 963, 1101, 1024, 816, 1063, 575, 1153, 447, 1117, 910, 1017, 931, 909, 1126, 1027, 871, 1052, 891, 375, 1131, 318, 705, 1048, 908, 1026, 1061, 865, 493, 742, 978, 575, 877, 339, 1078, 592, 1019, 1066, 717, 690, 800, 855, 1006, 1082, 614, 638, 1090, 441, 780, 764, 337, 871, 1005, 687, 1108, 205, 875, 1109, 395, 554, 600, 173, 860, 815, 816, 759, 1002, 708, 582, 786, 1135, 881, 572, 1033, 727, 1151, 613
The weights are: 94, 506, 416, 992, 649, 237, 457, 815, 446, 422, 791, 359, 667, 598, 7, 544, 334, 766, 994, 893, 633, 131, 428, 700, 617, 874, 720, 419, 794, 196, 997, 116,

--- temps de calcul (Programmation Dynamique) : 0.22600007057189941

--- temps de calcul (Branch & Bound: Borne 1) : 0.004999876022338867

--- temps de calcul (Branch & Bound: Borne 2) : 0.003000020980834961

test  : InstancesKnapSack/subset_sum/knapPI_6_100_1000_1_-997.opb.txt:

The prices are: 485, 94, 326, 506, 248, 416, 421, 992, 322, 649, 795, 237, 43, 457, 845, 815, 955, 446, 252, 422, 9, 791, 901, 359, 122, 667, 94, 598, 738, 7, 574, 544, 715, 334, 882, 766, 367, 994, 984, 893, 299, 633, 433, 131, 682, 428, 72, 700, 874, 617, 138, 874, 856, 720, 145, 419, 995, 794, 529, 196, 199, 997, 277, 116, 97, 908, 719, 539, 242, 707, 107, 569, 122, 537, 70, 931, 98, 726, 600, 487, 645, 772, 267, 513, 972, 81, 895, 943, 213, 58, 748, 303, 487, 764, 923, 536, 29, 724, 674, 789
The weights are: 485, 94, 326, 506, 248, 416, 421, 992, 322, 649, 795, 237, 43, 457, 845, 815, 955, 446, 252, 422, 9, 791, 901, 359, 122, 667, 94, 598, 738, 7, 574, 544, 715, 334, 882, 766, 367, 994, 984, 893, 299, 633

485, 326, 248, 421, 322, 795, 43, 845, 955, 252, 9, 901, 122, 94, 738, 574, 715, 882, 367, 984, 299, 433, 682, 72, 874, 138, 856, 145, 995, 529, 199, 277, 97, 719, 242, 107, 122, 70, 98, 600, 645, 267, 972, 895, 213, 748, 487, 923, 29, 674, 540, 554, 467, 46, 710, 553, 191, 724, 730, 988, 90, 340, 549, 196, 865, 678, 570, 936, 722, 651, 123, 431, 508, 585, 853, 642, 992, 725, 286, 812, 859, 663, 88, 179, 187, 619, 261, 846, 192, 261, 514, 886, 530, 849, 294, 799, 391, 330, 298, 790
The capacity is :995
--- temps de calcul (Programmation Dynamique) : 0.03399991989135742

--- temps de calcul (Branch & Bound: Borne 1) : 0.017000198364257812

--- temps de calcul (Branch & Bound: Borne 2) : 0.01699995994567871

test  : InstancesKnapSack/weakly_correlated_span/knapPI_12_100_1000_1_-970.opb.txt:

The prices are: 582, 194, 679, 485, 312, 873, 468, 208, 364, 260, 582, 388, 291, 104, 520, 416, 970, 260, 582, 364, 776, 194, 468, 970, 312, 260, 156, 485, 416, 364, 582, 388, 468, 679, 52, 156, 312,

The weights are: 94, 506, 416, 992, 649, 237, 457, 815, 446, 422, 791, 359, 667, 598, 7, 544, 334, 766, 994, 893
The capacity is :994
--- temps de calcul (Programmation Dynamique) : 0.011999845504760742

--- temps de calcul (Branch & Bound: Borne 1) : 0.002000093460083008

--- temps de calcul (Branch & Bound: Borne 2) : 0.002000093460083008

test  : InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt:

The prices are: 96, 507, 417, 993, 651, 237, 459, 816, 447, 423, 792, 360, 669, 600, 9, 546, 336, 768, 996, 894
The weights are: 94, 506, 416, 992, 649, 237, 457, 815, 446, 422, 791, 359, 667, 598, 7, 544, 334, 766, 994, 893
The capacity is :994
--- temps de calcul (Programmation Dynamique) : 0.0009999275207519531

--- temps de calcul (Branch & Bound: Borne 1) : 0.002000093460083008

--- temps de calcul (Branch & Bound: Borne 2) : 0.004999876022338867

test  : InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_3_-1002.opb.txt:

The prices are: 96, 507, 417, 993, 651, 237, 4

## Bonus 


In [46]:
function bellman_ford(graph, source)
    n = size(graph)[1]
    fk = zeros(n)
    fk_1 = zeros(n)
    predecessors = fill(0, n)

    for i in 1:n
        fk_1[i] = Inf
    end
    fk_1[source] = 0
    l = 0
    while l <= n
        l += 1
        for (i, j, k) in graph
            
            fk = copy(fk_1)
            if fk[i] + k < fk[j]
                fk_1[j] = fk[i] +k 
                predecessors[j] = i
            end 
        end 
        if fk == fk_1
            break
        end
    end
    return fk, predecessors
end 

# Exemple de graphe
graphe = [(1, 2, 3), (1, 5, 5), (5, 2, -1), (2, 3, 4), (3, 4, 2), (5, 4, 9), (5, 6, 3)]


f, pred = bellman_ford(graphe, 1)
println("Le plus court chemin entre A et F est A-B-C-D-F : ", f)
couple = ["A","B","C","D","E","F"] 
for i in 2:length(f)-1
    println(couple[1], " -> ", couple[i], " = ", f[i], " passant par ",couple[pred[i]])
end 

println("Ce chemin a un coût de 12 : ", pred)





Le plus court chemin entre A et F est A-B-C-D-F : [0.0, 3.0, 7.0, 9.0, 5.0, 8.0, Inf]
A -> B = 3.0 passant par A
A -> C = 7.0 passant par B
A -> D = 9.0 passant par C
A -> E = 5.0 passant par A
A -> F = 8.0 passant par E
Ce chemin a un coût de 12 : [0, 1, 2, 3, 1, 5, 0]
